# create shorter video from frames

In [ ]:
import cv2
import glob

In [ ]:
cv2.__version__

In [ ]:
frames = sorted(glob.glob("/root/data/pellet/frames/*"))[19:]

height , width , layers =  cv2.imread(frames[0]).shape
fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
# fourcc = VideoWriter_fourcc(*'MP4V')
video = cv2.VideoWriter("/root/data/pellet/videos/short.mp4",fourcc, 25, (width,height))

for f in frames:
    img = cv2.imread(f)
    video.write(img)

cv2.destroyAllWindows()
video.release()

# PREPARE ANNOTATIONS

In [1]:
import csv
import os

In [2]:
frame_bboxes = {}
start = 4120
with open('/root/data/pellet/labels/pellet_labels.csv', 'r') as f:
    reader = csv.reader(f)
    for l in reader:
        frame_number = int(os.path.basename(l[0]).split(".")[0].split("_")[-1]) - start
        if frame_number not in frame_bboxes:
            frame_bboxes[frame_number] = []
        frame_bboxes[frame_number].append((int(l[1]), int(l[2]), int(l[3])-int(l[1]), int(l[4])- int(l[2])))
        

# some utils function

In [3]:
from sklearn.metrics import pairwise_distances
import numpy as np

In [17]:
def merge_bboxes(previous, new, thresh=10.0):
    """ we have some annotations that we need to merge with the tracker"""
    print("merging bboxes")
    print("tracker bboxes: {}".format(len(previous)))
    print("detection bboxes: {}".format(len(new)))
    previous_centroids = [[(k[0]+k[2]) / 2.0, (k[1]+k[3]) / 2.0] for k in previous]
    new_centroids = [[(k[0]+k[2]) / 2.0, (k[1]+k[3]) / 2.0] for k in new]
    # print(len(previous))
    # calculate pairwise distances
    dist = pairwise_distances(np.array(previous_centroids), np.array(new_centroids))
    _, ys = np.where(dist < thresh)
    bad_index = sorted(list(set(ys)))
    # print(bad_index)
    true_new = []
    for (i, new_bb) in enumerate(new):
        # print(i)
        if i not in bad_index:
            true_new.append(new_bb)
    print("New bboxes: {}".format(len(true_new)))
    true_new = [(p[0], p[1], p[2], p[3]) for p in true_new]
    return dist, true_new

In [18]:
dist, merged = merge_bboxes(frame_bboxes[0], frame_bboxes[280])

merging bboxes
tracker bboxes: 24
detection bboxes: 74
New bboxes: 72


# pellet tracking

In [36]:
# import the necessary packages
from imutils.video import VideoStream
from imutils.video import FPS
import argparse
import imutils
import time
import cv2

In [37]:
tracker_name = "csrt"

In [38]:
OPENCV_OBJECT_TRACKERS = {
    "csrt": cv2.TrackerCSRT_create,
    "kcf": cv2.TrackerKCF_create,
    "boosting": cv2.TrackerBoosting_create,
    "mil": cv2.TrackerMIL_create,
    "tld": cv2.TrackerTLD_create,
    "medianflow": cv2.TrackerMedianFlow_create,
    "mosse": cv2.TrackerMOSSE_create
}

In [39]:
# tracker = cv2.TrackerCSRT_create


In [40]:
vs = cv2.VideoCapture("/root/data/pellet/videos/short.mp4")
fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
video = cv2.VideoWriter("/root/data/pellet/videos/test.mp4",fourcc, 25, (1920,1080))

In [41]:
fps = None

In [ ]:
# loop over frames from the video stream
i = 0
trackers = cv2.MultiTracker_create() 
bboxes = ()
while True:
    if i % 10 == 0: print("{}/1000".format(i))
    # grab the current frame, then handle if we are using a
    # VideoStream or VideoCapture object
    frame = vs.read()
    if i < 275:
        i += 1
        continue
        
    frame = frame[1] # if args.get("video", False) else frame
    height , width , layers = frame.shape
    
    # check to see if we have reached the end of the stream
    if frame is None:
        break
 
    (H, W) = frame.shape[:2]
    (success, bboxes) = trackers.update(frame)
    print(success)
    print("total bboxes: {}".format(len(bboxes)))
    
    if i in frame_bboxes:
        if len(bboxes) > 0:
            bboxes = [list(b) for b in bboxes]
            _, bboxes = merge_bboxes(bboxes, frame_bboxes[i])
            # print("new bboxes: {}".format(len(bboxes)))
        else:
            bboxes = frame_bboxes[i]
        # print(bboxes)
        for bbox in bboxes:
            tracker = OPENCV_OBJECT_TRACKERS[tracker_name]()
            trackers.add(tracker, frame, bbox)
            (x, y, w, h) = [int(v) for v in bbox]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    else:
        for bbox in bboxes:
            (x, y, w, h) = [int(v) for v in bbox]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
#     if i >= 275:
#    print(len(bboxes))
    video.write(frame)
    


    i += 1
    if i == 500:
        break
video.release()

0/1000
10/1000
20/1000
30/1000
40/1000
50/1000
60/1000
70/1000
80/1000
90/1000
100/1000
110/1000
120/1000
130/1000
140/1000
150/1000
160/1000
170/1000
180/1000
190/1000
200/1000
210/1000
220/1000
230/1000
240/1000
250/1000
260/1000
270/1000
True
total bboxes: 0
True
total bboxes: 77
True
total bboxes: 77
True
total bboxes: 77
True
total bboxes: 77
280/1000
True
total bboxes: 77
merging bboxes
tracker bboxes: 77
detection bboxes: 74
New bboxes: 6
True
total bboxes: 83
True
total bboxes: 83
True
total bboxes: 83
True
total bboxes: 83
True
total bboxes: 83
True
total bboxes: 83
True
total bboxes: 83
True
total bboxes: 83
merging bboxes
tracker bboxes: 83
detection bboxes: 87
New bboxes: 17
True
total bboxes: 100
290/1000
True
total bboxes: 100
True
total bboxes: 100
True
total bboxes: 100
True
total bboxes: 100
True
total bboxes: 100
True
total bboxes: 100
True
total bboxes: 100
True
total bboxes: 100
True
total bboxes: 100
True
total bboxes: 100
300/1000
True
total bboxes: 100
True
total

In [ ]:
video.release()